# Introduction

Chess has fascinated humanity for centuries, not only as a game but as a powerful medium for cognitive development. In recent years, the intersection of chess with artificial intelligence and computer vision has opened new possibilities for automated chess systems. This blog explores one such innovative approach from the paper "[Development of an autonomous chess robot system using computer vision and deep learning](https://www.sciencedirect.com/science/article/pii/S2590123025001793)" by Truong Duc Phuc and Bui Cao Son.

The paper talks about building an autonomous chess robot which sees and understands chessboard just like a human player. This involves two main tasks
1. Detecting the chessboard
2. Recognizing the chess pieces

## Chessboard detection:

![Pipeline from the Paper](image1.png)

### Canny Edge detection: (Finding the Borders of Squares)
- It takes the chessboard image as input and converts it into grayscale
- A Gaussian blur is applied to smooth the image and reduce noise
- Then Canny algorithm is used to detect edges of the squares
 
### Line Detection: (Finding the Grid Lines)
- The edges detected by `Canny Edge Detection` are processed using the `Hough Line Transform` algorithm to identify straight lines
- It identifies both vertical and horizontal grid lines that make up the board
 
### Finding Intersection Points: (Corners of Squares)
- It looks for where the lines intersect to find the corners of each square
- A clustering algorithm groups nearby lines to avoid duplicates (e.g., multiple lines detected for the same edge). See Fig 7
 
### Flattening the Chessboard: (Correcting Perspective)
- If the camera isn’t perfectly overhead, the chessboard might look tilted or distorted
- The Homography Transform fixes this by warping the image to make it look like a perfect top down view

![Clustering of lines](image2.png)

# Creation of Dataset

We downloaded the dataset from Kaggle ("https://www.kaggle.com/datasets/imtkaggleteam/chess-pieces-detection-image-dataset"). This dataset has ~1300 images.

We performed following actions on the dataset:
- Remove first unused Classname from `data.yaml` file
- Cleaned useless images
- Converted `green and white` chessboard images to `black and white` images
- Created new images with `_bnw` appended to the original filename in the same folder
- Modified all three folders `train`, `valid`, and `test`

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import uuid
import shutil

def change_to_black_and_white(image_path, output_path):

    img_cv = cv2.imread(image_path)
    hls = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HLS)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([100, 255, 255])

    hsv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    hls_modified = hls.copy()
    hls_modified[mask_green > 0, 2] = 0
    hls_modified[mask_green > 0, 0] = 0
    result_bgr = cv2.cvtColor(hls_modified, cv2.COLOR_HLS2BGR)
    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)
    Image.fromarray(result_rgb).save(output_path)

folders=['train','valid','test']

for folder in folders:
    image_folder = f'Chess_pieces/{folder}/images'
    txt_folder = f'Chess_pieces/{folder}/labels'
    ann=pd.read_csv(f'Chess_pieces/{folder}/_annotations.csv')
    files_name = [f for f in os.listdir(image_folder)]

    for file_name in files_name:
        original_image_path = os.path.join(image_folder, file_name)
        original_txt_path = os.path.join(txt_folder, file_name.replace('.jpg', '.txt'))
        if not os.path.exists(original_txt_path):
            continue
        new_name = str(uuid.uuid4())
        new_image_path = os.path.join(image_folder, new_name+'.jpg')
        new_txt_path = os.path.join(txt_folder, new_name+'.txt')

        new_name_bnw = os.path.join(image_folder, new_name+'_bnw.jpg')
        change_to_black_and_white(original_image_path, new_name_bnw)
        new_name_bnw_txt = os.path.join(txt_folder, new_name+'_bnw.txt')
    
        os.rename(original_image_path, new_image_path)
        os.rename(original_txt_path, new_txt_path)
        shutil.copyfile(new_txt_path, new_name_bnw_txt)
        ann.loc[ann['filename'] == file_name, 'filename'] = new_name+'.jpg'
        bnw_row = ann.loc[ann['filename'] == new_name+'.jpg'].copy()
        bnw_row['filename'] = new_name+'_bnw.jpg'
        ann = pd.concat([ann, bnw_row], ignore_index=True)

    ann.to_csv(f'Chess_pieces/{folder}/_annotations.csv', index=False)
    print(f'Folder {folder} done')